In [2]:
import sqlite3
import pandas as pd

# Reconnect to database
conn = sqlite3.connect("ecommerce.db")

# Replace these filenames with the actual ones you're using
csv_files = {
    "customers": "customers.csv",
    "sellers": "sellers.csv",
    "geolocation": "geolocation.csv",
    "product_category_name_translation": "product_category_name_translation.csv",
    "products": "products.csv",
    "orders": "orders.csv",
    "order_items": "order_items.csv",
    "payments": "order_payments.csv",
    "reviews": "order_reviews.csv"
}

# Import data
for table_name, csv_file in csv_files.items():
    print(f"📥 Importing {csv_file} into {table_name}...")
    df = pd.read_csv(csv_file)
    df.to_sql(table_name, conn, if_exists='append', index=False)

print("✅ All CSVs imported successfully.")
conn.close()


📥 Importing customers.csv into customers...
📥 Importing sellers.csv into sellers...
📥 Importing geolocation.csv into geolocation...
📥 Importing product_category_name_translation.csv into product_category_name_translation...
📥 Importing products.csv into products...
📥 Importing orders.csv into orders...
📥 Importing order_items.csv into order_items...
📥 Importing order_payments.csv into payments...
📥 Importing order_reviews.csv into reviews...
✅ All CSVs imported successfully.


In [3]:
#Basic Analysis
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()



In [4]:
# Query 1: Total Number of Customers

query1 = "SELECT COUNT(*) AS total_customers FROM customers;"
cursor.execute(query1)
print("Total Customers:", cursor.fetchone()[0])

Total Customers: 397764


In [5]:
#Query 2: Top 5 Product Categories by Number of Orders

query2 = """
SELECT p.product_category_name, COUNT(oi.order_id) AS total_orders
FROM order_items oi
JOIN products p ON oi.product_id = p.product_id
GROUP BY p.product_category_name
ORDER BY total_orders DESC
LIMIT 5;
"""
df2 = pd.read_sql_query(query2, conn)
print("\nTop 5 Categories by Orders:\n", df2)



Top 5 Categories by Orders:
     product_category_name  total_orders
0         cama_mesa_banho        177840
1            beleza_saude        154720
2           esporte_lazer        138256
3        moveis_decoracao        133344
4  informatica_acessorios        125232


In [6]:
#Query 3: Monthly Sales Trend

query3 = """
SELECT 
    strftime('%Y-%m', o.order_purchase_timestamp) AS order_month,
    SUM(oi.price + oi.freight_value) AS total_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
GROUP BY order_month
ORDER BY order_month;
"""
df3 = pd.read_sql_query(query3, conn)
print("\nMonthly Sales Trend:\n", df3)



Monthly Sales Trend:
    order_month   total_sales
0      2016-09  7.095000e+03
1      2016-10  1.136177e+06
2      2016-12  3.924000e+02
3      2017-01  2.743770e+06
4      2017-02  5.725612e+06
5      2017-03  8.640972e+06
6      2017-04  8.248445e+06
7      2017-05  1.172382e+07
8      2017-06  1.005926e+07
9      2017-07  1.169943e+07
10     2017-08  1.336409e+07
11     2017-09  1.440798e+07
12     2017-10  1.538625e+07
13     2017-11  2.358288e+07
14     2017-12  1.727094e+07
15     2018-01  2.214604e+07
16     2018-02  1.973818e+07
17     2018-03  2.310254e+07
18     2018-04  2.319396e+07
19     2018-05  2.299564e+07
20     2018-06  2.045354e+07
21     2018-07  2.117456e+07
22     2018-08  2.006617e+07
23     2018-09  3.329200e+03


In [7]:
# Query 4: Customer Order Count

query4 = """
SELECT c.customer_unique_id, COUNT(o.order_id) AS order_count
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_unique_id
ORDER BY order_count DESC
LIMIT 10;
"""
df4 = pd.read_sql_query(query4, conn)
print("\nCustomer Order Count:\n", df4)



Customer Order Count:
                  customer_unique_id  order_count
0  8d50f5eadf50201ccdcedfb9e2ac8455          340
1  3e43e6105506432c953e165fb2acf44c          180
2  ca77025e7201e3b30c44b472ff346268          140
3  6469f99c1f9dfae7733b25662e7f1782          140
4  1b6c7548a2a1f9037c1fd3ddfed95f33          140
5  f0e310a6839dce9de1638e0fe5ab282a          120
6  de34b16117594161a6a89c50b289d35a          120
7  dc813062e0fc23409cd255f7f53c7074          120
8  63cfc61cee11cbe306bff5857d00bfe4          120
9  47c1a3033b8b77b3ab6e109eb4d5fdf3          120


In [8]:
# Query 5: Total Payment per Order by Payment Type

query6 = """
SELECT order_id, payment_type, SUM(payment_value) AS total_payment
FROM payments
GROUP BY order_id, payment_type
ORDER BY total_payment DESC
LIMIT 10;
"""
df6 = pd.read_sql_query(query6, conn)
print("\nTotal Payment per Order:\n", df6)




Total Payment per Order:
                            order_id payment_type  total_payment
0  03caa2c082116e1d31e67e9ae3700499  credit_card       40992.24
1  736e1922ae60d0d6a89247b851902527       boleto       21824.64
2  0812eb902a67711a1cb742b3cdaa65ae  credit_card       20787.93
3  fefacc66af859508bf1a7934eab1e97f       boleto       20766.63
4  f5136e38d1a14a4dbd87dff67da82701       boleto       20179.98
5  2cc9089445046817a7539d90805e6e5a       boleto       18244.62
6  a96610ab360d42a2e5335a3998b4718a  credit_card       14851.02
7  b4c4b76c642808cbe472a32b86cddc95  credit_card       14428.32
8  199af31afc78c699f0dbf71fb178d4d4  credit_card       14293.02
9  8dbc85d1447242f3b127dda390d56e19  credit_card       14045.34
